In [1]:
pip install qiskit qiskit-aer numpy matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 4.1 MB/s eta 0:00:00


In [33]:
print("QFT: Transforms quantum states into frequency domain")
print("IQFT: Inverse transform to extract information")
print("Phase Estimation: Estimates eigenvalue phases")
print("Shor's Algorithm: Factors integers using period finding")

QFT: Transforms quantum states into frequency domain
IQFT: Inverse transform to extract information
Phase Estimation: Estimates eigenvalue phases
Shor's Algorithm: Factors integers using period finding


In [19]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from math import pi, gcd

In [28]:
simulator = AerSimulator()

In [26]:
# QFT Implementation
def qft_circuit(n_qubits):
    qc = QuantumCircuit(n_qubits)

    for i in range(n_qubits):
        qc.h(i)
        for j in range(i+1, n_qubits):
            qc.cp(pi/2**(j-i), i, j)

    # Swap qubits to reverse order
    for i in range(n_qubits//2):
        qc.swap(i, n_qubits-1-i)

    return qc

qc = QuantumCircuit(3, 3)
qc.x(0)  # |1⟩
qc.x(2)  # |1⟩
qc.append(qft_circuit(3), range(3))
qc.measure_all()

job = simulator.run(transpile(qc, simulator), shots=1000)
result = job.result()
counts = result.get_counts()
print("QFT Results:")
for state, count in sorted(counts.items()):
    print(f"|{state}⟩: {count/1000:.3f}")

QFT Results:
|000 000⟩: 0.122
|001 000⟩: 0.122
|010 000⟩: 0.151
|011 000⟩: 0.123
|100 000⟩: 0.124
|101 000⟩: 0.117
|110 000⟩: 0.115
|111 000⟩: 0.126


In [48]:
# IQFT Implementation
def iqft_circuit(n_qubits):
    qc = QuantumCircuit(n_qubits)

    # Reverse the swaps first
    for i in range(n_qubits//2):
        qc.swap(i, n_qubits-1-i)

    # Reverse the QFT operations
    for i in range(n_qubits-1, -1, -1):
        for j in range(n_qubits-1, i, -1):
            qc.cp(-pi/2**(j-i), i, j)
        qc.h(i)

    return qc

print("IQFT Implementation")

qc = QuantumCircuit(3, 3)
qc.x(0)  # Start with |100⟩
qc.append(qft_circuit(3), range(3))
qc.append(iqft_circuit(3), range(3))
qc.measure_all()

job = simulator.run(transpile(qc, simulator), shots=1000)
result = job.result()
counts = result.get_counts()
print("IQFT Results (should recover |100⟩)")
for state, count in sorted(counts.items()):
    print(f"|{state}⟩: {count/1000:.3f}")



IQFT Implementation
IQFT Results (should recover |100⟩)
|001 000⟩: 1.000


In [44]:
# Phase Estimation Implementation
def phase_estimation_circuit(n_counting_qubits=3):
    qc = QuantumCircuit(n_counting_qubits + 1, n_counting_qubits)


    for i in range(n_counting_qubits):                                          # Initialize counting qubits in superposition
        qc.h(i)

    # Prepare eigenstate |1⟩ (eigenstate of Z with eigenvalue -1)
    qc.x(n_counting_qubits)

    # Apply controlled-Z operations
    for j in range(n_counting_qubits):
        qc.cz(j, n_counting_qubits)

    # Apply IQFT to counting qubits
    iqft = iqft_circuit(n_counting_qubits)
    qc.append(iqft, range(n_counting_qubits))

    # Measure counting qubits
    qc.measure(range(n_counting_qubits), range(n_counting_qubits))

    return qc

qc = phase_estimation_circuit(3)

job = simulator.run(transpile(qc, simulator), shots=1000)
result = job.result()
counts = result.get_counts()
print("Phase Estimation Results ")
for state, count in sorted(counts.items()):
    decimal_value = int(state, 2)
    estimated_phase = decimal_value / 8
    print(f"|{state}⟩ = {count/1000:.3f}, estimated phase {estimated_phase:.3f}")
print("Expected: phase = 0.5 for eigenvalue -1 of Z gate")


Phase Estimation Results 
|100⟩ = 0.081, estimated phase 0.500
|101⟩ = 0.457, estimated phase 0.625
|110⟩ = 0.400, estimated phase 0.750
|111⟩ = 0.062, estimated phase 0.875
Expected: phase = 0.5 for eigenvalue -1 of Z gate


In [47]:
# Shor's Algorithm Implementation
def shors_period_finding(N=15, a=7):
    n_qubits = 4
    qc = QuantumCircuit(2*n_qubits, n_qubits)

    # Initialize first register in superposition
    for i in range(n_qubits):
        qc.h(i)

    # Apply controlled modular exponentiation
    for i in range(n_qubits):
        if i % 2 == 0:
            qc.cx(i, n_qubits + i % n_qubits)

    # Apply QFT to first register
    qft = qft_circuit(n_qubits)
    qc.append(qft, range(n_qubits))

    # Measure
    qc.measure(range(n_qubits), range(n_qubits))

    return qc
print("Shor's Algorithm Implementation")

N = 15
a = 7

print(f"Factoring N = {N} using a = {a}")
print(f"Period verification: {a}^4 mod {N} = {(a**4) % N}")

qc = shors_period_finding(N, a)

job = simulator.run(transpile(qc, simulator), shots=1000)
result = job.result()
counts = result.get_counts()
print("Quantum Period Finding Results")
for state, count in sorted(counts.items()):
    if count > 50:
        decimal_value = int(state, 2)
        print(f"|{state}⟩= {count/1000:.3f}, decimal - {decimal_value}")

Shor's Algorithm Implementation
Factoring N = 15 using a = 7
Period verification: 7^4 mod 15 = 1
Quantum Period Finding Results
|0000⟩= 0.260, decimal - 0
|0010⟩= 0.117, decimal - 2
|0011⟩= 0.133, decimal - 3
|1000⟩= 0.116, decimal - 8
|1011⟩= 0.079, decimal - 11
|1100⟩= 0.076, decimal - 12
|1111⟩= 0.119, decimal - 15


In [43]:
# Classical post-processing
r = 4  # Period is 4
factor1 = gcd(a**(r//2) - 1, N)
factor2 = gcd(a**(r//2) + 1, N)
print(f"Classical post-processing:")
print(f"gcd({a}^{r//2} - 1, {N}) = {factor1}")
print(f"gcd({a}^{r//2} + 1, {N}) = {factor2}")
if factor1 > 1 and factor1 < N:
    print(f"Found factors: {factor1} × {N//factor1} = {N}")
elif factor2 > 1 and factor2 < N:
    print(f"Found factors: {factor2} × {N//factor2} = {N}")

Classical post-processing
gcd(7^2 - 1, 15) = 3
gcd(7^2 + 1, 15) = 5
Found factors: 3 × 5 = 15
